In [4]:
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, ImageClip, CompositeVideoClip, AudioFileClip
import pyttsx3 as tts

def speak(text):
    engine = tts.init()
    engine.save_to_file(text, 'samp.mp3')
    engine.runAndWait()
    

def get_files_in_directory(directory):
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

def generate_video():
    with open('samp.txt', 'r') as file:
        text = file.read()
    
    lst = text.split(' ')

    clips = []

    for i in range(len(lst)):
        txt_data = TextClip(lst[i], fontsize=70, color='white', bg_color='none', font='Arial-Bold')
        img_data = ImageClip('Z_1.jpg')
        img_data = img_data.set_duration(0.5)
        txt_data = txt_data.set_duration(0.5)
        txt_data = txt_data.set_position('center')
        img_data = img_data.set_position('center')


        clipped = CompositeVideoClip([img_data, txt_data], size=(1920, 1080))
        clips.append(clipped)

    

    final = concatenate_videoclips(clips)
    audio = AudioFileClip("samp.mp3")
    final = final.set_audio(audio)
    final.write_videofile("output0.mp4", fps=24, codec = 'libx264')

In [6]:
with open('samp.txt', 'r') as file:
    text = file.read()

speak(text)

In [7]:
generate_video()

Moviepy - Building video output0.mp4.
MoviePy - Writing audio in output0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output0.mp4



Moviepy - Done !
Moviepy - video ready output0.mp4
